<a href="https://colab.research.google.com/github/Nidhig19/NLP/blob/main/notebooks/TestCaseNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install spacy gensim
!pip install tabulate

In [2]:
!git clone https://github.com/Nidhig19/NLP.git

Cloning into 'NLP'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 115 (delta 31), reused 69 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 1.21 MiB | 2.49 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [3]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import spacy, gensim
from tabulate import tabulate
from spacy import displacy
nlp = spacy.load('en_core_web_md')

# Data Preprocessing

In [5]:
#Import and read file
with open('NLP/data/sample.txt') as file:
    sample = file.read()
text = nlp(sample)
sentence_spans = list(text.sents)

In [6]:
for sentence in sentence_spans:
    print(sentence)

As a UI designer, I want to report to the Agencies about user testing, so that they are aware of their contributions to making Broker a better UX.

As a Researcher, I want an app that create proxy Data Packages for well know and reliable data, sources, so that I can load high quality data using Data Package tooling. 

As a participant, I want to change my estimate as long as the draw has not been completed, so that I can change my mind.

As a depositor, I want to have metadata automatically filled from other University systems and remembered from previous deposits, so that I don't have to waste time reentering the same information.



In [7]:
#Removing punctuations, stop words, whitespaces
sentence_tokens=[]
for i,sentence in enumerate(sentence_spans):
    filtered_text=[token for token in sentence if not token.is_punct and not token.is_stop and not token.is_space]
    sentence_tokens.append(filtered_text)
    print(i+1,filtered_text)

1 [UI, designer, want, report, Agencies, user, testing, aware, contributions, making, Broker, better, UX]
2 [Researcher, want, app, create, proxy, Data, Packages, know, reliable, data, sources, load, high, quality, data, Data, Package, tooling]
3 [participant, want, change, estimate, long, draw, completed, change, mind]
4 [depositor, want, metadata, automatically, filled, University, systems, remembered, previous, deposits, waste, time, reentering, information]


# Input, Action and Condition Words


In [8]:
from spacy.tokens import Doc

In [9]:
def custom_dep_tree(span, main_verb_index):
  words = [token.text for token in span]
  new_doc = Doc(span.vocab, words=words)

  for i, token in enumerate(span):
    new_doc[i].pos_ = token.pos_
    new_doc[i].lemma_ = token.lemma_
    new_doc[i].tag_ = token.tag_
    new_doc[i].dep_ = token.dep_
    head_index = min(token.head.i - span[0].i, len(new_doc) - 1)
    new_doc[i].head = new_doc[head_index]

  if main_verb_index is not None:
    # Set the main verb as root
    new_doc[main_verb_index].dep_ = "ROOT"
    new_doc[main_verb_index].head = new_doc[main_verb_index]

  # Adjust dependencies for other tokens
  for token in new_doc:
      if token.i != main_verb_index and token.dep_ == "ROOT":
          token.dep_ = "dep"
          token.head = new_doc[main_verb_index]

  return new_doc


In [10]:
res = []
for id,tokens in enumerate(sentence_tokens):
  main_verb_index = None
  results = {}
  for i, token in enumerate(tokens):

    if token.lemma_ == "want":
      for j, next_token in enumerate(tokens[i:], start=i):
        if (next_token.pos_ == "VERB" and
          next_token.lemma_ not in ["want", "be", "have"] and
          not next_token.dep_ == "aux"):
          main_verb_index = j
          break

      if main_verb_index:
        break

  if main_verb_index is not None:

    main_verb_text = tokens[main_verb_index].text
    orig_main_verb_index = next(
        (i for i, token in enumerate(sentence_spans[id])
         if token.text == main_verb_text),
        None
    )

    modified_doc = custom_dep_tree(sentence_spans[id], orig_main_verb_index)
    main_verb = modified_doc[orig_main_verb_index]
    # print(main_verb)


    # Role Extraction
    role = None
    for i, token in enumerate(sentence_spans[id]):
        if token.text.lower() == "as" and i + 2 < len(sentence_spans[id]):
            # Collect role words
            role_words = []
            for j in range(i + 2, len(sentence_spans[id])):
                current_token = sentence_spans[id][j]
                if current_token.text.lower() == "," and j + 1 < len(sentence_spans[id]) and sentence_spans[id][j+1].text.lower() == "i":
                    break
                role_words.append(current_token.text)

            role = " ".join(role_words)
            break


    # Condition Extraction
    condns = None
    for i, token in enumerate(sentence_spans[id]):
        if (token.text.lower() == "so"):
            # Collect condition words
            condn_words = []
            for j in range(i, len(sentence_spans[id])):
                current_token = sentence_spans[id][j]
                if current_token.text.lower() == ".":
                  break
                condn_words.append(current_token.text)

            condns = " ".join(condn_words)
            break


  results = {
          'main_action': main_verb.text,
          'role': role or "Unknown",
          'conditions': condns or "Unknown",
          'original_text': sentence_spans[id],
          'filtered_text':tokens,
          'main_verb_index': main_verb_index,
          'dependencies': [(token.text, token.dep_, token.head.text)
                          for token in modified_doc if token.i != orig_main_verb_index],
          'modified_doc': modified_doc
      }
  res.append(results)
  print("\n", results['original_text'])
  print("Main Action:", results['main_action'])
  print("Role:", results['role'])
  print("Conditions:", results['conditions'])
  print("Dependencies: ", results['dependencies'])


 As a UI designer, I want to report to the Agencies about user testing, so that they are aware of their contributions to making Broker a better UX.

Main Action: report
Role: UI designer
Conditions: so that they are aware of their contributions to making Broker a better UX
Dependencies:  [('As', 'prep', 'want'), ('a', 'det', 'designer'), ('UI', 'compound', 'designer'), ('designer', 'pobj', 'As'), (',', 'punct', 'want'), ('I', 'nsubj', 'want'), ('want', 'dep', 'report'), ('to', 'aux', 'report'), ('to', 'prep', 'report'), ('the', 'det', 'Agencies'), ('Agencies', 'pobj', 'to'), ('about', 'prep', 'report'), ('user', 'compound', 'testing'), ('testing', 'pobj', 'about'), (',', 'punct', 'want'), ('so', 'mark', 'are'), ('that', 'mark', 'are'), ('they', 'nsubj', 'are'), ('are', 'advcl', 'want'), ('aware', 'acomp', 'are'), ('of', 'prep', 'aware'), ('their', 'poss', 'contributions'), ('contributions', 'pobj', 'of'), ('to', 'prep', 'contributions'), ('making', 'pcomp', 'to'), ('Broker', 'nsubj', 

# Dependency Parsing

In [11]:
for i,result in enumerate(res):
    data = []
    displacy.render(result['modified_doc'], style="dep")
    filtered_words = [token.text for token in result['filtered_text']]
    filtered_tokens = [token for token in result['modified_doc'] if token.text in filtered_words]
    for token in filtered_tokens:
        data.append([token.text, token.lemma_, token.pos_, token.tag_, token.dep_])

    # Print the table
    headers = ["Text", "Lemma", "POS", "Tag", "Dependency"]
    print(tabulate(data, headers=headers, tablefmt="fancy_grid"))

╒═══════════════╤══════════════╤═══════╤═══════╤══════════════╕
│ Text          │ Lemma        │ POS   │ Tag   │ Dependency   │
╞═══════════════╪══════════════╪═══════╪═══════╪══════════════╡
│ UI            │ ui           │ NOUN  │ NN    │ compound     │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ designer      │ designer     │ NOUN  │ NN    │ pobj         │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ want          │ want         │ VERB  │ VBP   │ dep          │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ report        │ report       │ VERB  │ VB    │ ROOT         │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ Agencies      │ agency       │ NOUN  │ NNS   │ pobj         │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ user          │ user         │ NOUN  │ NN    │ compound     │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ testing       │ testing      │ NOUN  │

╒════════════╤════════════╤═══════╤═══════╤══════════════╕
│ Text       │ Lemma      │ POS   │ Tag   │ Dependency   │
╞════════════╪════════════╪═══════╪═══════╪══════════════╡
│ Researcher │ researcher │ NOUN  │ NN    │ pobj         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ want       │ want       │ VERB  │ VBP   │ dep          │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ app        │ app        │ NOUN  │ NN    │ dobj         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ create     │ create     │ VERB  │ VBP   │ ROOT         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ proxy      │ proxy      │ ADJ   │ JJ    │ compound     │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ Data       │ Data       │ PROPN │ NNP   │ compound     │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ Packages   │ Packages   │ PROPN │ NNP   │ dobj         │
├────────────┼────────────┼───────┼───────┼─────────────

╒═════════════╤═════════════╤═══════╤═══════╤══════════════╕
│ Text        │ Lemma       │ POS   │ Tag   │ Dependency   │
╞═════════════╪═════════════╪═══════╪═══════╪══════════════╡
│ participant │ participant │ NOUN  │ NN    │ pobj         │
├─────────────┼─────────────┼───────┼───────┼──────────────┤
│ want        │ want        │ VERB  │ VBP   │ dep          │
├─────────────┼─────────────┼───────┼───────┼──────────────┤
│ change      │ change      │ VERB  │ VB    │ ROOT         │
├─────────────┼─────────────┼───────┼───────┼──────────────┤
│ estimate    │ estimate    │ NOUN  │ NN    │ dobj         │
├─────────────┼─────────────┼───────┼───────┼──────────────┤
│ long        │ long        │ ADV   │ RB    │ advmod       │
├─────────────┼─────────────┼───────┼───────┼──────────────┤
│ draw        │ draw        │ NOUN  │ NN    │ nsubjpass    │
├─────────────┼─────────────┼───────┼───────┼──────────────┤
│ completed   │ complete    │ VERB  │ VBN   │ advcl        │
├─────────────┼─────────

╒═══════════════╤═══════════════╤═══════╤═══════╤══════════════╕
│ Text          │ Lemma         │ POS   │ Tag   │ Dependency   │
╞═══════════════╪═══════════════╪═══════╪═══════╪══════════════╡
│ depositor     │ depositor     │ NOUN  │ NN    │ pobj         │
├───────────────┼───────────────┼───────┼───────┼──────────────┤
│ want          │ want          │ VERB  │ VBP   │ dep          │
├───────────────┼───────────────┼───────┼───────┼──────────────┤
│ metadata      │ metadata      │ PROPN │ NNP   │ dobj         │
├───────────────┼───────────────┼───────┼───────┼──────────────┤
│ automatically │ automatically │ ADV   │ RB    │ advmod       │
├───────────────┼───────────────┼───────┼───────┼──────────────┤
│ filled        │ fill          │ VERB  │ VBN   │ ROOT         │
├───────────────┼───────────────┼───────┼───────┼──────────────┤
│ University    │ University    │ PROPN │ NNP   │ compound     │
├───────────────┼───────────────┼───────┼───────┼──────────────┤
│ systems       │ system 

# Transformers Setup

In [12]:
!pip install accelerate transformers
!pip install -U bitsandbytes
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# Import Llama 3.3

In [2]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [3]:
model_name = "meta-llama/Llama-3.3-8B-Instruct"

In [4]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

## Quantization Configuration

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## Loading Tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, token = HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    token=HF_TOKEN)

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

model-00001-of-00030.safetensors:   0%|          | 0.00/4.58G [00:00<?, ?B/s]

model-00002-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00003-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00012-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00013-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00014-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00015-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

KeyboardInterrupt: 